In [1]:
import os

In [2]:
# Sample config for test experiments
BASEDIR='/media/1tb_drive/multilingual-multimodal/flickr30k/train/processed/BERTHA-TEST_Adam_wmt-multimodal_internal_data_'+\
    'dropout0.3_ff_noiseFalse_search_model_en2es_vocab20000_emb300_rec800_batch15/'

exp_config = {
    'src_vocab_size': 20000,
    'trg_vocab_size': 20000,
    'enc_embed': 300,
    'dec_embed': 300,
    'enc_nhids': 800,
    'dec_nhids': 800,
#     'saved_parameters': '/home/chris/projects/neural_mt/archived_models/BERTHA-TEST_Adam_wmt-multimodal_internal_data_dropout'+\
#     '0.3_ff_noiseFalse_search_model_en2es_vocab20000_emb300_rec800_batch15/best_bleu_model_1455464992_BLEU31.61.npz',
    'src_vocab': os.path.join(BASEDIR, 'vocab.en-de.en.pkl'),
    'trg_vocab': os.path.join(BASEDIR, 'vocab.en-de.de.pkl'),
    'src_data': os.path.join(BASEDIR, 'training_data/train.en.tok.shuf'),
    'trg_data': os.path.join(BASEDIR, 'training_data/train.de.tok.shuf'),
    
    'unk_id':1,
    # Bleu script that will be used (moses multi-perl in this case)
    'bleu_script': '/home/chris/projects/neural_mt/test_data/sample_experiment/tiny_demo_dataset/multi-bleu.perl',
    # Optimization related ----------------------------------------------------
    # Batch size
    'batch_size': 5,
    # This many batches will be read ahead and sorted
    'sort_k_batches': 1,
    # Optimization step rule
    'step_rule': 'AdaDelta',
    # Gradient clipping threshold
    'step_clipping': 1.,
    # Std of weight initialization
    'weight_scale': 0.01,
    'seq_len': 40,
    'finish_after': 100,
    
    # WORKING: add config for evaluation with IMT model -- follow NMT evaluation config

    ##############                                                                                  
    # PREDICTION #                                                                                              
    ##############      
    'prefix_decoding': True,
    'brick_delimiter': '-',
    'beam_size': 10,
    'normalized_bleu': True,

    # Note we _do not_ need the target_prefixes, since these are generated on the fly

    'source_lang': 'en',                                                                                                    
    'target_lang': 'de',                                                                                                                 

    'n_best': 1,                                                                                                                                 

    # path to the moses perl script for tokenization                                                                                    
    'tokenize_script': None,                                                                                                                                
    # path to the moses perl script for detokenization                                                                                              
    'detokenize_script': None,                                                                                                                              

    # The location of the saved parameters of a trained model as .npz                                                                       
    # TODO: model save directory is currently misnamed -- switch to yaml configs with good model names                          
    'saved_parameters': '/media/1tb_drive/multilingual-multimodal/flickr30k/train/processed/BERTHA-TEST_Adam_wmt-multimodal_internal_data_'+\
    'dropout0.3_ff_noiseFalse_search_model_en2es_vocab20000_emb300_rec800_batch15/'+\
    'best_bleu_model_1455464992_BLEU31.61.npz',
                                                                                                                                                           
                                                                                                                                                        
    # The location of a test set in the source language                                                                                                             
    'test_set': '/media/1tb_drive/multilingual-multimodal/flickr30k/test/test.en.tok',                                   
    #'test_set': '/media/1tb_drive/multilingual-multimodal/flickr30k/train/processed/dev.en.tok'                                                        
                                                                                                                            
    # your desired path to the translated output file, or an already-translated file that you just want to evaluate                                           
    'translated_output_file': '/media/1tb_drive/multilingual-multimodal/flickr30k/train/processed/test.imt_PROTO-DELETE-ME.30.x.hyps.out',         
    #'translated_output_file': '/media/1tb_drive/multilingual-multimodal/flickr30k/train/processed/dev.multimodal-summed.30.x.hyps.out'             
                                                                                                                                                            
    # The location of the gold standard references for the test set (for evaluation mode only)                                                  
    'test_gold_refs': '/media/1tb_drive/multilingual-multimodal/flickr30k/test/test.de.tok', 
    
    # if the config contains this key, meteor will also be computed                                                                                                          │
    'meteor_directory': '/home/chris/programs/meteor-1.5'   
}

In [3]:
# Test a fuel stream which maps (source, target) into (source, target, target_prefix, target_suffix)

from nn_imt.stream import (PrefixSuffixStreamTransformer, CopySourceAndTargetToMatchPrefixes)

from machine_translation.stream import ShuffleBatchTransformer
                           

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 4007)


In [4]:
import numpy

from fuel.datasets import IterableDataset
from fuel.transformers import Merge
from fuel.streams import DataStream
from fuel.datasets import TextFile
from fuel.schemes import ConstantScheme
from fuel.streams import DataStream
from fuel.transformers import (
    Merge, Batch, Filter, Padding, SortMapping, Unpack, Mapping)
import numpy
from fuel.datasets import TextFile
from fuel.schemes import ConstantScheme
from fuel.streams import DataStream
from fuel.transformers import (
    Merge, Batch, Filter, Padding, SortMapping, Unpack, Mapping, Transformer)

from six.moves import cPickle

from machine_translation.stream import _ensure_special_tokens, _length, PaddingWithEOS, _oov_to_unk, _too_long

# Working: modify this for IMT
def get_tr_stream_with_prefixes(src_vocab, trg_vocab, src_data, trg_data, src_vocab_size=30000,
                                trg_vocab_size=30000, unk_id=1, seq_len=50,
                                batch_size=80, sort_k_batches=12, **kwargs):
    """Prepares the training data stream."""

    # Load dictionaries and ensure special tokens exist
    src_vocab = _ensure_special_tokens(
        src_vocab if isinstance(src_vocab, dict)
        else cPickle.load(open(src_vocab)),
        bos_idx=0, eos_idx=src_vocab_size - 1, unk_idx=unk_id)
    trg_vocab = _ensure_special_tokens(
        trg_vocab if isinstance(trg_vocab, dict) else
        cPickle.load(open(trg_vocab)),
        bos_idx=0, eos_idx=trg_vocab_size - 1, unk_idx=unk_id)

    # Get text files from both source and target
    src_dataset = TextFile([src_data], src_vocab, None)
    trg_dataset = TextFile([trg_data], trg_vocab, None)

    # Merge them to get a source, target pair
    stream = Merge([src_dataset.get_example_stream(),
                    trg_dataset.get_example_stream()],
                   ('source', 'target'))

    # Filter sequences that are too long
    stream = Filter(stream,
                    predicate=_too_long(seq_len=seq_len))

    # Replace out of vocabulary tokens with unk token
    # TODO: doesn't the TextFile stream do this anyway?
    stream = Mapping(stream,
                     _oov_to_unk(src_vocab_size=src_vocab_size,
                                 trg_vocab_size=trg_vocab_size,
                                 unk_id=unk_id))

    # 
    stream = Mapping(stream, PrefixSuffixStreamTransformer(),
                     add_sources=('target_prefix', 'target_suffix'))
    
    # WORKING: add transformer to duplicate the source and target len(target_prefix) times
    stream = Mapping(stream, CopySourceAndTargetToMatchPrefixes(stream))
    
    # changing stream.produces_examples is a little hack which lets us use Unpack to flatten
    stream.produces_examples = False
    # flatten the stream back out into (source, target, target_prefix, target_suffix)
    stream = Unpack(stream)
    
    # Now make a very big batch that we can shuffle
    # TODO: let user configure the size of this shuffle batch
    # Build a batched version of stream to read k batches ahead
    SHUFFLE_BATCH_SIZE = 1000
    stream = Batch(stream,
                   iteration_scheme=ConstantScheme(SHUFFLE_BATCH_SIZE)
                  )
    
    # WORKING: implement shuffled batch transformer in nnmt
    stream = ShuffleBatchTransformer(stream)
    
    # unpack it again
    stream = Unpack(stream)


    # Build a batched version of stream to read k batches ahead
    stream = Batch(stream,
                   iteration_scheme=ConstantScheme(batch_size*sort_k_batches)
                  )

    # Sort all samples in the read-ahead batch
    stream = Mapping(stream, SortMapping(_length))

    # Convert it into a stream again
    stream = Unpack(stream)

    # Construct batches from the stream with specified batch size
    stream = Batch(
        stream, iteration_scheme=ConstantScheme(batch_size))

    print(src_vocab_size)
    # Pad sequences that are short
    # TODO: is it correct to blindly pad the target_prefix and the target_suffix?
    masked_stream = PaddingWithEOS(
        stream, [src_vocab_size - 1, trg_vocab_size - 1, trg_vocab_size - 1, trg_vocab_size - 1],
        mask_sources=('source', 'target', 'target_prefix', 'target_suffix'))

    return masked_stream, src_vocab, trg_vocab

In [5]:
test_tr_stream, src_vocab, trg_vocab = get_tr_stream_with_prefixes(**exp_config)

20000


In [6]:
t = test_tr_stream.get_epoch_iterator()

In [7]:
source,source_mask,target,target_mask,target_prefix,target_prefix_mask,target_suffix,target_suffix_mask = t.next()

In [8]:
trg_ivocab = {v:k for k,v in trg_vocab.items() if k != '<S>'}

In [9]:
for pre, suf in zip(target_prefix, target_suffix):
    print(pre)
    print('prefix: {}'.format(' '.join([trg_ivocab[int(w)] for w in pre])))
    print('suffix: {}'.format(' '.join([trg_ivocab[int(w)] for w in suf])))

[    3    10     6 19999 19999 19999 19999 19999 19999 19999 19999 19999
 19999 19999 19999]
prefix: Ein Mann , </S> </S> </S> </S> </S> </S> </S> </S> </S> </S> </S> </S>
suffix: der im Wasser paddelt . </S> </S> </S>
[    3   760    22   465  6378     6  5475    18  1284 19999 19999 19999
 19999 19999 19999]
prefix: Ein Team von vier Richtern , bewertet einen Wettbewerb </S> </S> </S> </S> </S> </S>
suffix: . </S> </S> </S> </S> </S> </S> </S>
[    3    10     5 19999 19999 19999 19999 19999 19999 19999 19999 19999
 19999 19999 19999]
prefix: Ein Mann in </S> </S> </S> </S> </S> </S> </S> </S> </S> </S> </S> </S>
suffix: einem grauen Oberteil jongliert mit Kegeln . </S>
[   12   277    15     8    11  8553  3494  2145     7 19999 19999 19999
 19999 19999 19999]
prefix: Eine blonde Frau mit einer 40er Jahre Frisur und </S> </S> </S> </S> </S> </S>
suffix: Leopardenmuster Hemd . </S> </S> </S> </S> </S>
[   3   67   25    8  202    9    4 5513  444   14   34    8 3037  357    2]
prefix

In [10]:
test_tr_stream.sources

('source',
 'source_mask',
 'target',
 'target_mask',
 'target_prefix',
 'target_prefix_mask',
 'target_suffix',
 'target_suffix_mask')